# EGB103 Assignment One - Wake Modeling and Control for Wind Farms

Programming and software play a crucial role in aiding engineers in problem-solving tasks related to design, layout and control of wind turbines within large-scale wind farms using methods such as computational fluid dynamics and wake modeling. These tools enable engineers to efficiently process complex mathematical calculations, simulate dynamic behaviors, and visualize results. Through programming, engineers can automate repetitive tasks, implement sophisticated numerical methods and find optimal solutions to engineering design problems.

In this assignment, to help us learn Python in the context of an Authentic Engineering problem, we will use a Python Module called FLORIS to try to optimally position and orient the wind turbines within a wind farm and steer their wakes to further optimize energy output given known local prevailing wind patterns. You may study the theory and mathematics behind the kind of analysis performed by Floris in later units such as EGB323 Fluid Mechananics.

<div style="background-color: #00ccff; padding:10px">
    
## Part A - Due Friday 9th August 2024 (end of week 3)

- Task 1 - *Add markdown*
- Task 5 - *Rotate a single location*
- Task 10 - *Compute the distance downwind for a particular turbine*

## Part B - Due Friday 30th August 2024 (end of week 6)

- Task 2 - *Creating a layout of turbines within a wind farm*
- Task 3 - *Implement create hexagonal layout*
- Task 4 - *Which layout produces the best energy production?*
- Task 6 - *Rotate an entire layout*
- Task 7 - *Finding the best rotation*
- Task 8 - *Which layout is now the best?*
- Task 9 - *Greedily optimize the yaw angle for just one turbine*
- Task 11 - *Order by downwind distance*
- Task 12 - *Optimize all yaw angles*

</div>

<div style="background-color: #00cc00; padding:10px">

# Task 1: Add markdown to list your name and previous programming experience
    
Create a new markdown cell ***immediately below this cell*** that includes your name and briefly describes your previous programming experience if any.

Must include some markdown to style or format in some manner.
</div>


<div style="background-color: #ffcc00; padding:10px">

# Rules and Restrictions

- You must use Jupyter in the cloud (on https://jupyter.eres.qut.edu.au) to develop your solution.
- For the assignments you cannot work with friends or colleagues, or get help from anyone other than the EGB103 teaching team - it needs to be entirely your own work.
- You cannot use AI tools such as ChatGPT or Copilot to help develop your solution.
- Do not modify the name or list of parameters for any of the functions.
- Every function must be implemented in only one place within this .ipynb file. Do not cut and paste to duplicate function definitions.
- The only python module that you are allowed to import is the math module. The Floris, Numpy and Matplotlib modules are also imported but should only be used within the functions that I have provided for you, not  within the functions that you need to implement.
- When importing the math module, it should be just `import math`. Do not use other forms of import such as `from <module_name> import <name(s)>` or `import <module_name> as <alt_name>`. This means that all use of features within the math module will be explicit, e.g. `math.sin(0.5)`.
- You should only use Python language features that we have covered in lectures or tutorials, for example you should not use match statements or list comprehensions (don't worry if you don't know what these are).

</div>

In [ ]:
import code_analyser

ok = code_analyser.test_rules_and_restrictions()

# Wake Modeling

The prevailing wind direction (and speed) will vary from day to day and even hour to hour. Wind turbines are normally designed to turn so that they are always facing towards the prevailing wind direction as this maximizes the wind velocity over the blades and so maximizes rotor rotation speed and therefore energy production. 

Once the air has passed through the rotating blades of a wind turbine, its velocity will tend to decrease and the amount of turbulance will tend to increase. This wake affect can have a negative effect on other nearby wind turbines located down wind. The turbulance also causes variations in the mechanical loading on the blades which can cause long term structural fatique. Even at a distance of 500 metres, this wake effect is still very evident. We could place all of the wind turbines very far apart, so as to eliminate any wake effect, however that would require more land resources and also more cabling and time spent travelling between turbines during maintainance. The following diagram shows an example of these wakes when the wind is coming from the south west (as modeled and visualized by Floris).

<img src="images/wake.png" style="display: block; margin: auto" />


# Wind Rose

A wind rose is a graphic tool used by meteorologists to give a succinct view of how wind speed and direction are typically distributed at a particular location.

https://en.wikipedia.org/wiki/Wind_rose

The Floris Python module provides support for creating a Wind Rose for a particular location as a Python data object.

The following function (which has been provided for you) creates a Wind Rose based on data provided by the Australian Bureau of Metrorology for the Brisbane Area.

http://www.bom.gov.au/climate/averages/wind/selection_map.shtml

For example, in Brisbane the wind is from the North (0 degrees) with a speed of around 1 metre/second on average 3.4% of the time (over the course of a year).

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import floris
import numpy

# This function creates a wind rose for Brisbane Australia.
# Parameters:
# - None
# Return value:
# - a Floris WindRose object
# Makes use of floris

def brisbane_wind_rose() :
    standard_wind_directions = [0,45,90,135,180,225,270,315]
    standard_wind_speeds = [1,4,7,10,13]
    brisbane_wind_freqs = [[3.4,3.0,0.7,0.0,0.0],[1.7,1.8,0.3,0.0,0.0],[1.8,3.2,1.3,0.2,0.2],[3.0,4.6,2.4,0.3,0.3],[8.8,12.0,2.3,0.1,0.1],[10.7,11.9,2.4,0.2,0.2],[2.7,2.4,1.7,0.5,0.5],[3.0,1.3,0.2,0.0,0.0]]
    return floris.WindRose(wind_directions=numpy.array(standard_wind_directions), 
                                wind_speeds=numpy.array(standard_wind_speeds), 
                                ti_table=0.06, 
                                freq_table = numpy.array(brisbane_wind_freqs))

In [ ]:
ax = brisbane_wind_rose().plot(ws_step=3)

# Representing the location of each Wind Turbine within the Wind Farm

We will represent the location of each wind turbine within the Wind Farm using 2D coordinates which will be represented as a Python tuple containing 2 numbers.
The first number is the x coordinate (or its position in the east-west direction), while the second number is the y coordinate (or its position in the north-south direction).

The layout of the entire wind farm will therefore be represented by a list of 2D coordinates (i.e. a list of tuples).

For example, a wind farm containing 4 wind turbines might have a layout of say [(0,0), (250, 500), (500, 0), (750, 500)] 

However, internally the Floris model likes to represent the wind farm layout as a list of x coordinates and a list of y coordinates. So internally the above layout would be represented as x coordinates: [0, 250, 500, 750] and y coordinates [0, 500, 0, 500].

The following function (which has been provided for you) converts a list of 2D coordinates to a list of x coordinates and a list of y coordinates.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

# This function converts a list of 2D coordinates to a list of x coordinates and a list of y coordinates.
# Parameters:
# - layout is a list of (x,y) tuples (corresponding to the location of each turbine)
# Return value:
# - a tuple containing a list of the x coordinates and a list of the y coordinates.

def unzip(layout) :
    x_coords = []
    y_coords = []
    for (x, y) in layout :
        x_coords.append(x)
        y_coords.append(y)
    return x_coords, y_coords

In [ ]:
import code_analyser

x_coords1, y_coords1 = unzip([(0,0), (250, 500), (500, 0), (750, 500)])
code_analyser.assert_equal(x_coords1, [0, 250, 500, 750])
code_analyser.assert_equal(y_coords1, [0, 500, 0, 500])

# Modeling the Energy produced by the Wind Farm

The Floris module provides means to estimate the total power produced by the wind farm per hour. To do this it uses the wind rose for the area where the wind farm is located. For each combination of wind direction and wind speed included in the wind rose it computes the estimated power produced by each of the wind turbines taking account of the affect of wind speed and turbulance that may result from the wake generated by other turbines upwind. The power produced by the farm is the sum of the power produced by each wind turbine within the farm. The average power produced is computed as a weighted average overall all the different combinations of prevailing wind speed and direction based on the frequency of each combination as captured in the wind rose for the local area.

The following function (which has been provided for you) computes the average kilowatts of energy produced by the farm per hour (we divide the power measured in watts by 1000 to compute the power measured in kilowatts. In your subsequent functions you must always use this function to estimate the power produced for a given layout of the turbines - you must not use the FlorisModel or any of its functions/methods directly yourself.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import floris

# This function takes the location of each turbine in a wind farm and models the expected power generated by the entire wind farm assuming Brisbane wind patterns.
# Parameters:
# - layout is a list of (x,y) tuples (corresponding to the location of each turbine)
# Return value:
# - a tuple containg a pair of values: the Floris model object, and the expected power generated by the entire farm measured in kilowatts per hour.
# Makes use of unzip, brisbane_wind_rose, floris

def evaluate_turbine_layout(layout) :
    
    x_coords, y_coords = unzip(layout)
    fmodel = floris.FlorisModel("gch.yaml") # read from configuration file gch.yaml which provides specifications for the wind turbines
    fmodel.set(layout_x=x_coords, layout_y=y_coords)
    
    fmodel.set(wind_data=brisbane_wind_rose())

    fmodel.run()      
    return fmodel, fmodel.get_expected_farm_power() / 1000 # divide by 1000 to convert watts to kilowatts

In [ ]:
model2, power2 = evaluate_turbine_layout([(0,0), (250, 500), (500, 0), (750, 500)]) 
code_analyser.assert_equal(power2, 1249.2909419363739)

# Visualizing the Layout of wind turbines within the Wind farm

The following function (which has been provided for you) allows you to easily visualise the layout of each the wind  turbines within a wind farm.
For each wind turbine it plots a dot with  a label. The visualization also includes the total average power generated per hour based on  a specific layout of turbines within the wind farm.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import floris.layout_visualization
import matplotlib

# This function provides a visualisation the layout of each the turbines within a wind farm. For each turbine it plots a dot with a label. 
# The visualization also includes the total average power generated per hour based on a specific layout of turbines within the wind farm.
# Parameters:
# - layout is a list of (x,y) tuples (corresponding to the location of each turbine)
# Return value:
# - None (does not return a value)
# Makes use of: evaluate_turbine_layout, floris, matplotlib

def visualize_layout(layout) :
    fmodel, power = evaluate_turbine_layout(layout)
    figure, axes = matplotlib.pyplot.subplots()
    
    figure.suptitle(f"Entire grid of turbines produces {power:.2f} kW")
    
    floris.layout_visualization.plot_turbine_points(fmodel, ax=axes)
    floris.layout_visualization.plot_turbine_labels(fmodel, ax=axes)

In [ ]:
visualize_layout([(0,0), (250, 500), (500, 0), (750, 500)]) 

<div style="background-color: #00cc00; padding:10px" id="fred">

# Task 2: Creating a layout of turbines within a wind farm

The simplest manner in which to lay out turbines within a wind farm is to arrange them neatly into a set of equally spaced rows and columns.

The following function (which you need to implement) takes as input the number of rows and columns and the distance between these rows and columns, and produces a layout, i.e. a list of tuples (x,y) coordinates corresponding to the location of each of the turbines within the wind farm.

The first turbine is always located at location (0,0), then we have the other turbines located in the first column, followed by the turbines in the second column and so on.

</div>

In [ ]:
# This function creates simple grid layout where the turbines are organised neatly into evenly spaced rows and columns.
# The first turbine is always located at location (0,0), then we have the other turbines located in the first column, followed by the turbines in the second column and so on.
# Parameters:
# - number_of_rows is an integer number that specifies how many equally spaced rows of turbines to create
# - number_of_columns is an integer number that specifies how  many equally spaced columns of turbines to create
# - minimum_gap_between_turbines is the gap (measured in metres) between all rows and columns of turbines 
# Return value:
# - a list of (x,y) tuples corresponding to the x and y coordinates of each of the turbines. 

def create_grid_layout(number_of_rows, number_of_columns, minimum_gap_between_turbines) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

    # Hint: look at the doubly nested loop example used to print the 12 times tables

In [ ]:
import code_analyser

grid21 = create_grid_layout(2, 1, 100)
code_analyser.assert_equal(grid21, [(0, 0), (0, 100)], 'test1a')

In [ ]:
grid12 = create_grid_layout(1, 2, 50)
code_analyser.assert_equal(grid12, [(0, 0), (50, 0)], 'test1b')

In [ ]:
grid35 = create_grid_layout(3, 5, 100)
code_analyser.assert_equal(grid35, [(0, 0), (0, 100), (0, 200), (100, 0), (100, 100), (100, 200), (200, 0), (200, 100), (200, 200), (300, 0), (300, 100), (300, 200), (400, 0), (400, 100), (400, 200)], 'test1c')

In [ ]:
grid43 = create_grid_layout(4, 3, 500)
code_analyser.assert_equal(grid43, [(0, 0), (0, 500), (0, 1000), (0, 1500), (500, 0), (500, 500), (500, 1000), (500, 1500), (1000, 0), (1000, 500), (1000, 1000), (1000, 1500)], 'test1d')

In [ ]:
grid_layout43a = create_grid_layout(4, 3, 500)
if grid_layout43a:
    visualize_layout(grid_layout43a)

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_create_grid_layout()

# Alternative Hexagonal Layout

If we wish to maintain a regular lattice structure, but wish to pack the turbines more densely while still maintaining a mimimum distance (M) between all wind turbines, then the optimal arrangement is a hexagonal packing (see https://en.wikipedia.org/wiki/Circle_packing#Densest_packing).

In the illustration below, the red dots represent the locations of the wind turbines. The blue circles (which have a radius equal to the minimum allowed distance between turbines) demonstrate that none of the wind turbines are closer together than this minimum distance. The green lines highlight the hexagonal nature of this arrangement. 

Note that the distance between the rows is M, but the distance between the columns is less than M. This example has 6 rows and 8 columns.

<img src="images/hex.png">

<div style="background-color: #00cc00; padding:10px">
    
# Task 3: Implement create hexagonal layout

The following function (which you need to implement) creates one of these hexagonal layouts, with the specified number of rows and columns and minimum distance between turbines. You may assume that there are an even number of columns.

As with the grid layout, the first turbine is always at location (0,0), followed by the other turbines in the first column, then the turbines in the next column and so on.

</div>

In [ ]:
# This function creates a hexagonal layout where the turbines are packed as closely together as possible.
# As with the grid layout, the first turbine is always at location (0,0), followed by the other turbines in the first column, then the turbines in the next column and so on.
# Parameters:
# - number_of_rows is an integer number that specifies how many turbines there are in each column
# - number_of_columns is an integer number that specifies how  many equally spaced columns of turbines to create (you may assume that there are an even number of columns)
# - minimum_gap_between_turbines is the minimum gap (measured in metres) between all turbines
# Return value:
# - a list of (x,y) tuples corresponding to the x and y coordinates of each of the turbines. 

import math

def create_hexagonal_layout(number_of_rows, number_of_columns, minimum_gap_between_turbines) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

In [ ]:
import code_analyser

test_layout22 = create_hexagonal_layout(2, 2, 500)
code_analyser.assert_equal(test_layout22, [(0.0, 0), (0.0, 500), (433.0127018922193, 250.0), (433.0127018922193, 750.0)], 'test2a')

In [ ]:
test_layout34 = create_hexagonal_layout(3, 4, 500)
code_analyser.assert_equal(test_layout34, [(0.0, 0), (0.0, 500), (0.0, 1000), (433.0127018922193, 250.0), (433.0127018922193, 750.0), (433.0127018922193, 1250.0), (866.0254037844386, 0), (866.0254037844386, 500), (866.0254037844386, 1000), (1299.0381056766578, 250.0), (1299.0381056766578, 750.0), (1299.0381056766578, 1250.0)], 'test2b')

In [ ]:
test_layout44 = create_hexagonal_layout(4, 4, 500)
code_analyser.assert_equal(test_layout44, [(0.0, 0), (0.0, 500), (0.0, 1000), (0.0, 1500), (433.0127018922193, 250.0), (433.0127018922193, 750.0), (433.0127018922193, 1250.0), (433.0127018922193, 1750.0), (866.0254037844386, 0), (866.0254037844386, 500), (866.0254037844386, 1000), (866.0254037844386, 1500), (1299.0381056766578, 250.0), (1299.0381056766578, 750.0), (1299.0381056766578, 1250.0), (1299.0381056766578, 1750.0)], 'test2c')

In [ ]:
test_layout33 = create_hexagonal_layout(3, 3, 100)
code_analyser.assert_equal(test_layout33, [(0.0, 0), (0.0, 100), (0.0, 200), (86.60254037844386, 50.0), (86.60254037844386, 150.0), (86.60254037844386, 250.0), (173.20508075688772, 0), (173.20508075688772, 100), (173.20508075688772, 200)], 'test2d')

In [ ]:
hex_layout34 = create_hexagonal_layout(3, 4, 500)
if hex_layout34:
    visualize_layout(hex_layout34)

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_create_hexagonal_layout()

<div style="background-color: #00cc00; padding:10px">
    
# Task 4: Which layout produces the best energy production?

Which produces the best energy production, a grid layout or a hexagonal layout (assuming the same total number of turbines with the same number of rows and columns)?

Comment on your observations here.

# Rotating a layout

If the wind is often from say the south or the west, then the problem with a grid layout (and to a slightly less extent a hexagonal layout) 
is that many of the turbines will be positioned directly downwind of other turbines, which will reduce wind velocity and increase turbulance 
reaching those downwind turbines and so decrease their energy production. A simple solution to this problem might be to orient the wind farm so that it is not oriented directly north-south/east-west.
We can experiment with orienting the wind farm at different angles and seek to find the optimal angle given the variety of prevailing wind directions that are most commonly encountered according to the wind rose.

To create a new re-oriented layout, we can take any given layout and rotate the location of every wind turbine by $d$ degree (clockwise) around the origin (0,0).

For example, if we rotated a 4 x 3 grid layout by 30 degrees (clockwise):

<img src="images/rotate.png">

<div style="background-color: #00cc00; padding:10px">
    
# Task 5: Rotate a single location

The first step towards rotating an entire layout is to be able to rotate a single point/ turbine location. The following function (which you need to implement) returns the new location of the specified location, after it has been rotated by the specified number of degrees (clockwise) about the origin.

The mathematics related to such a geometric transform is not overly complex, but is not expected to be known to you. We therefore expect you, as an engineer in training, to be able research/lookup this kind of well defined technical challenge. It this case it should be as simple as a Google search for "*formula for rotating a point around the origin*". Note: what you are seeking is a mathematical formula, not Python code. Once you have a mathematical formula you should then be able to translate it into a Python expression based on the function parameters provided. 

<img src="images/rotateLocation.png" width="400">

</div>

In [ ]:
import math

# This function computes the location that will result from rotating a given location about the origin by a specified angle (clockwise) measured in degrees.
# Parameters:
# - location is an (x,y) tuple representing the location that we wish to rotate
# - angle_degrees is the angle (measured in degrees) that we wish to rotate the location
# Return value:
# - An (x,y) tuple that represents the location that results from rotating the given location (around the origin)

def rotate_single_location(location, angle_degrees) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

    # Hint: to unpack the location tuple into its constituent x and y components use: 
    # x, y = location

In [ ]:
import code_analyser

new_position1 = rotate_single_location((0,0), 0)
code_analyser.assert_equal(new_position1, (0,0), "test3a")

In [ ]:
new_position2 = rotate_single_location((0,0), 42)
code_analyser.assert_equal(new_position2, (0,0), "test3b")

In [ ]:
new_position3 = rotate_single_location((1,1), 0)
code_analyser.assert_equal(new_position3, (1,1), "test3c")

In [ ]:
new_position4 = rotate_single_location((1,1), 45)
code_analyser.assert_equal(new_position4, (1.414213562373095, 0), "test3d")

In [ ]:
new_position5 = rotate_single_location((1,1), 10)
code_analyser.assert_equal(new_position5, (1.1584559306791384, 0.8111595753452777), "test3e")

In [ ]:
new_position6 = rotate_single_location((2,-1), 30)
code_analyser.assert_equal(new_position6, (1.2320508075688774, -1.8660254037844386), "test3f")

In [ ]:
new_position7 = rotate_single_location((1,1), 90)
code_analyser.assert_equal(new_position7, (1,-1), "test3g")

In [ ]:
new_position8 = rotate_single_location((1,1), 135)
code_analyser.assert_equal(new_position8, (0, -1.414213562373095), "test3h")

In [ ]:
new_position9 = rotate_single_location((1,1), 180)
code_analyser.assert_equal(new_position9, (-1, -1), "test3i")

In [ ]:
new_position10 = rotate_single_location((1,1), 225)
code_analyser.assert_equal(new_position10, (-1.4142135623730951, 0), "test3j")

In [ ]:
new_position11 = rotate_single_location((1,1), 270)
code_analyser.assert_equal(new_position11, (-1, 1), "test3k")

In [ ]:
new_position12 = rotate_single_location((12.6,-17.5), 15.6)
code_analyser.assert_equal(new_position12, (7.429751480883344, -20.243734658711368), "test3m")

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_rotate_single_location()

<div style="background-color: #00cc00; padding:10px">
    
# Task 6: Rotate an entire layout

We can now use the rotate_single_location function to help us rotate each of the locations within our original layout.

Note that we are not modifying/changing the original layout, we are creating a brand new separate layout (i.e. a new Python list).
</div>

In [ ]:
# This function computes the location that will result from rotating a given location about the origin by a specified angle (clockwise) measured in degrees.
# Important node: we are not modifying the original layout, we are creating a new layout (i.e. a new list) that would be the result of rotating the original layout. 
# Parameters:
# - layout is a list of (x,y) tuples representing the initial locations of each turbine
# - angle_degrees is the angle (measured in degrees) that we wish to rotate the layout
# Return value:
# - a new list of (x,y) tuples that presents the layout of the turbines if they were all rotated the specified angle about the origin.
# Makes use of: rotate_single_location

def rotate_layout(layout, angle_degrees) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

In [ ]:
import code_analyser

new_layout3 = rotate_layout([(0,0),(1,1), (2,2)], 45)
code_analyser.assert_equal(new_layout3,  [(0.0, 0.0), (1.414213562373095, 0), (2.82842712474619, 0)], "test4a")

In [ ]:
new_layout4 = rotate_layout([(0,0),(0,100),(0,200),(100,0),(100,100),(100,200),(200,0),(200,100),(200,200),(300,0),(300,100),(300,200),(400,0),(400,100),(400,200)], 0)
code_analyser.assert_equal(new_layout4, [(0,0),(0,100),(0,200),(100,0),(100,100),(100,200),(200,0),(200,100),(200,200),(300,0),(300,100),(300,200),(400,0),(400,100),(400,200)], "test4b")

In [ ]:
new_layout5 = rotate_layout([(0,0),(0,100),(0,200),(100,0),(100,100),(100,200),(200,0),(200,100),(200,200),(300,0),(300,100),(300,200),(400,0),(400,100),(400,200)], 45)
code_analyser.assert_equal(new_layout5, [(0.0, 0.0), (70.71067811865474, 70.71067811865476), (141.42135623730948, 141.4213562373095), (70.71067811865476, -70.71067811865474), (141.4213562373095, 1.4210854715202004e-14), (212.13203435596424, 70.71067811865477), (141.4213562373095, -141.42135623730948), (212.13203435596427, -70.71067811865473), (282.842712474619, 2.842170943040401e-14), (212.13203435596427, -212.13203435596424), (282.842712474619, -141.42135623730948), (353.5533905932738, -70.71067811865473), (282.842712474619, -282.84271247461896), (353.5533905932738, -212.1320343559642), (424.26406871192853, -141.42135623730945)], "test4c")

In [ ]:
new_layout6 = rotate_layout([(12.6,4.5), (-10.6,17.7), (-22.6,27.8)], 135.5)
code_analyser.assert_equal(new_layout6, [(-5.832863969993356, -12.04108375137194), (19.966548739141686, -5.19489474845059), (35.604737698420365, -3.9878131133096115)], "test4d")

In [ ]:
grid_layout43b = create_grid_layout(4, 3, 500)
rotated_grid_layout2 = rotate_layout(grid_layout43b, 30)
if rotated_grid_layout2:
    visualize_layout(rotated_grid_layout2)

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_rotate_layout()

<div style="background-color: #00cc00; padding:10px">

# Task 7: Finding the best rotation

Now that we can create new rotated layouts and we can evalulate the average power produced per hour for each such layout, we are now in a position to search for the best possible rotation, i.e. the rotation that yield the maximum power.

In the following function (which you need to implement), we will test every possible rotation of the original layout provided, ranging from -45 degrees to +45 degrees. To save time, we won't consider every possible angle in that range, but instead just steps of 5 degrees. So in total we will explore -45, -40, -35, -30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40 and 45 degrees. For each we will evaluate the power that would be produced for that rotated layout and return the layout that yields the highest average power.
</div>

In [ ]:
# This function tests every possible rotation of the original layout provided, ranging from -45 degrees to +45 degrees (in steps of 5).
# For each we will evaluate the power that would be produced for that rotated layout and return the layout that yields the highest average power.
# Parameters:
# - layout is a list of (x,y) tuples representing the initial locations of each turbine
# Return value:
# - a list of (x,y) tuples that presents the layout of the turbines if they were all rotated from their original location by the optimal angle (between -45 and +45 degrees)
# Makes use of: rotate_layout, evaluate_turbine_layout

def find_best_rotation(layout) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

    # Hint: when debugging, placing the following print statement inside your loop my help you understand the calculations as they are performed
    # print(f"rotating {angle} degrees produces {power:.2f} kW")

In [ ]:
import code_analyser

rotated_grid2 = find_best_rotation([(0, 0), (0, 500), (0, 1000), (500, 0), (500, 500), (500, 1000), (1000, 0), (1000, 500), (1000, 1000)])
code_analyser.assert_equal(rotated_grid2, [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)], "test5a")

In [ ]:
rotated_grid3 = find_best_rotation([(-62.9785,425.1580),(-294.4899,-46.8077),(123.5603,-405.6379),(122.6698,-310.7602)])
code_analyser.assert_equal(rotated_grid3,[(-135.8496269935961, 407.7627928979676), (-281.8878648980094, -97.23422033698607), (192.12142353215484, -378.01932790861156), (174.76911252186775, -284.73766706285755)], "test5b")

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_find_best_rotation()

In [ ]:
# Find the best rotated 4x4 regular grid layout ...

grid_layout_4x4 = create_grid_layout(4, 4, 500)
best_grid_4x4 = find_best_rotation(grid_layout_4x4)
if best_grid_4x4:
    visualize_layout(best_grid_4x4)

In [ ]:
# Find the best rotated 4x4 hexagonal layout ...

hex_layout_4x4 = create_hexagonal_layout(4, 4, 500)
best_hex_4x4a = find_best_rotation(hex_layout_4x4)
if best_hex_4x4a:
    visualize_layout(best_hex_4x4a)

<div style="background-color: #00cc00; padding:10px">

# Task 8: Which layout is now the best?

If we consider the affect of rotation, which now produces the best overall power, the optimally rotated grid layout or the optimally rotated hexagonal layout?

Why do you think that might be?

Include your observations and comments here.

# After positioning each wind turbine

In the following we will be assuming that we have selected the location for each wind turbine (via the above or other means) and will be seeking to further improve the power yield from the farm with changing any of those locations ...

# Wake Steering

Normally a wind turbine is designed to turn so that it faces directly towards the current prevailing wind direction. However, it is also possible to instead point the turbine either slight to the left or right of where the wind is coming from. This is referred to as the ***yaw*** angle - the angle between the direction of the wind and the direction that the turbine is facing. This will have a negative effect on the power generated by that particular wind turbine, however, by changing the angle of the wind turbine relative to the wind, we can also change the direction of the *wake* caused by that turbine. In particular, we can actively ***steer*** the wake so as to try to avoid or lessen the wake affect on other nearby wind turbines which are downwind. So, the reduced negative wake effects on the downwind turbines can actually more than compensate for the slightly lower efficiency of the upwind turbine (which doesn't point directly towards the wind). Each wind turbine within the wind farm can be independently steered, either left or right any number of degrees. It is therefore a complex optimization problem to determine the best *yaw* angle for each of the turbines. And when the wind changes direction we have entirely new/different optimization problem which needs to be rapidly solved so as to maintain optimal efficiency of the wind farm as a whole.

| Without Wake Steering | With Wake Steering |
| --------------------- | ------------------ |
| <img src="images/wake.png"> | <img src="images/steering.png"> |

# Representing yaw angles as a list

To describe the wake steering we will use a list of yaw angles, one for each wind turbine. For example, if we have 16 wind turbines, then the following yaw angle list [$-16$, $-12$, $-12$, $0$, $-12$, $-12$, $0$, $0$, $-16$, $-12$, $-12$, $0$, $0$, $0$, $0$, $0$] tells us that the first wind turbine will have a yaw angle of $-16$ degrees, while the second wind turbine will have a yaw angle of $-12$ degrees.

Without active wake steering, the yaw angle for each wind turbine will be zero. The following function (which has been provided for you) creates a list of yaw angles for the simple special case where all wind turbines are directly facing the prevailing wind, i.e. all yaw angles are zero.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

# This function returns a list of yaw angles (one for each turbine) with all angles equal to zero.
# Parameters:
# - layout is a list of (x,y) tuples (corresponding to the location of each turbine)
# Return value:
# - A list of the yaw angles for each turbine

def zero_yaw(layout) :
    return [0] * len(layout)

In [ ]:
import code_analyser

layout03 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
yaw_angles03 = zero_yaw(layout03)
code_analyser.assert_equal(yaw_angles03, [0, 0, 0, 0, 0, 0, 0, 0, 0])

# Evaluating a particular wind direction

Whenever we wish to perform wake steering, we need to know the prevailing wind direction since different wind directions will require different wake steering (with different yaw angles).

The following function (which has been provided for you) computes the estimated total power from the wind farm given a layout, a given wind direction and a given set of yaw angles.
We don't yet know which set of yaw angles will produce the maximum total power for a given wind direction, however, this function will help us compare different sets of yaw angles and thereby help us find the best.


In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import floris

# This function models the expected power generated by the entire wind farm given the location of each turbine, the prevailing wind direction and the specified yaw angle for each turbine.
# The wind speed is assumed to be 7.
# Parameters:
# - layout is a list of (x,y) tuples corresponding to the location of each turbine
# - wind_direction is the angle from which the wind is blowing, measured in degrees clockwise from North.
# - yaw_angles is a list of the yaw angles for each turbine
# Return value:
# - a tuple containg a pair of values: the Floris model object, and the expected power generated by the entire farm measured in kilowatts per hour.

def evaluate_single_direction_with_yaws(layout, wind_direction, yaw_angles) :
    x_coords, y_coords = unzip(layout)
    fmodel = floris.FlorisModel("gch.yaml")
    fmodel.set(layout_x=x_coords, layout_y=y_coords)
    
    fmodel.set(wind_directions=[wind_direction], wind_speeds=[7], turbulence_intensities=[0.06], yaw_angles=numpy.array([yaw_angles], dtype=float))   
    
    fmodel.run()
    return fmodel, fmodel.get_expected_farm_power() / 1000 

In [ ]:
import code_analyser

layout04 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
no_steering = zero_yaw(layout04)
model04, power04 = evaluate_single_direction_with_yaws(layout04, 225, no_steering)
code_analyser.assert_equal(power04, 9791.825293281678)

# Visualizing the Wake

The following function (which has been provided for you), provides a graphical visualization (from above) of the wake that results from a given layout, wind direction and yaw angles for each turbine.


In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import floris

# This function provides a graphical visualization (from above) of the wake that results from a given layout, wind direction and yaw angles for each turbine.
# Parameters:
# - layout is a list of (x,y) tuples corresponding to the location of each turbine
# - wind_direction is the angle from which the wind is blowing, measured in degrees clockwise from North.
# - yaw_angles is a list of the yaw angles for each turbine
# - axis is an optimal parameter specifying which matplotlib subplot (if any) the visualization should be displayed within
# Return value:
# - the total estimated power produced by the wind farm given this configuration.
# Makes use of: evaluate_single_direction_with_yaws

def visualize_flow_with_yaws(layout, wind_direction, yaw_angles, axis=None) :
    fmodel, power = evaluate_single_direction_with_yaws(layout, wind_direction, yaw_angles)
    horizontal_plane = fmodel.calculate_horizontal_plane(height=90.0)
    floris.flow_visualization.visualize_cut_plane(horizontal_plane, ax = axis)
    return power

In [ ]:
layout05 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
no_steering05 = zero_yaw(layout05)
visualize_flow_with_yaws(layout05, 225, no_steering05)

In [ ]:
layout06 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
wind_direction06 = 225 # wind from South West
yaw_angles06 = zero_yaw(layout06) # without any yaw steering
visualize_flow_with_yaws(layout06, wind_direction06, yaw_angles06)

In [ ]:
layout07 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
wind_direction07 = 135 # wind from South East
yaw_angles07 = zero_yaw(layout07) # without any yaw steering
visualize_flow_with_yaws(layout07, wind_direction07, yaw_angles07)

# Heuristic Solutions

Finding the perfect yaw angle for every turbine within a large wind farm is a computationally challenging problem, i.e. there are just so many different combinations of possible yaw angles that even a super fast computer able to perform billions of computation per second will still take too long to compute the optimal solution. There is little value in spending say 12 hours computing the optimal yaw angle for each turbine, because by then the wind may have shifted to a different direction.

For such computationally intensive calculations, it is common to use a heuristic, i.e. an simpler and faster algorithm that isn't guaranteed to always find the best possible solution, but will generally find pretty good solution. We will use such a heuristic to try to determine a yaw angle for each wind turbine that will maximize the overall energy production of the wind farm. The heuristic that we will use is referred to as a "greedy" heuristic. We start with a simple solution where every turbine has a yaw angle of zero degrees. i.e. it points directly towards the wind. We then consider each wind turbine one by one and for each turbine we will "greedily" choose the yaw angle for that wind turbine that yields the best overall energy production for the whole of the wind farm. We will then lock in that choice of yaw angle for that particular turbine (and will not reconsider it again) as we then move on to likewise greedily choose the locally optimal/best yaw angle for each of the remaining turbines. We say it is locally optimal meaning that given fixed yaw angles for all the other turbines, the one we select is best, even though some other combination of yaw angles for other turbines might produce an overall better solution.



<div style="background-color: #00cc00; padding:10px">

# Task 9: Greedily optimize the yaw angle for just one turbine

The following function (which you need to implement), takes as input a set of yaw angles (one for each turbine). Our goal is to find the optimal yaw angle for one particular wind turbine (given as the first parameter).
We can set the yaw angle for that turbine to whatever we like, but the yaw angles of all other other turbines need to remain as whatever they are specified to be in the input yaw_angles array.

There are an infinite number of possible real-valued angles to choose from (even for just one turbine), so we will instea just explore a finite sample of possible yaw angles for the turbine in question. We will only explore yaw angles between $-20$ degrees and $+20$ degrees, but even if we explored in 1 degree increment, exploring $41$ different angles will still take a bit longer than we would like, so to speed up our experiment, we will only explore in increments of $4$ degrees (i.e. just $-20$, $-16$, $-12$, $-8$, $-4$, $0$, $4$, $8$, $12$, $16$ and $20$).

To explore a particular angle for a our particular turbine, we will need to update the list of yaw angles and change the entry for just the turbine that we are focusing on. We can then call evaluate_single_direction_with_yaws to determine the total wind farm power that will result. In this way we can determine which of the various angles considered produces the best overall power.

This particular function is not designed to explicitly return a value. It will instead alter the yaw_angles list that is pasted as a parameter.
It should leave all the other yaw angles unaltered, but set the yaw angle for the turbine in question to the optimal angle (from amongst those that we considered).

For example, if the turbine_number is 3 and the input yaw angles are [$-16$, $-8$, $-12$, $0$, $-4$, $-12$, $0$, $0$, $-16$, $-12$, $-12$, $0$, $0$, $0$, $0$, $0$] then if the best yaw angle determined for turbine 3 is say $4$ degrees, then the greedy_optimize_yaw_angle_for_just_one_turbine function should alter that list so that on return it contains:
[$-16$, $-8$, $-12$, $4$, $-4$, $-12$, $0$, $0$, $-16$, $-12$, $-12$, $0$, $0$, $0$, $0$, $0$] 

</div>


In [ ]:
# This function greedily selects the optimal yaw angle for the specified turbine number.
# All yaw angle for all the other turbines needs to remain unaltered, but the entry for the specified turbine number will be updated/replaced by the optimal angle (between -20 and +20 degrees, in steps of 4).
# Important note: this function does not return any value, it instead updates/modifies the list of yaw angles that is passed to it as the last parameter.
# Parameters:
# - turbine_number is the number of the turbine for which we are going to select the best yaw angle 
# - layout is a list of (x,y) tuples corresponding to the location of each turbine
# - wind_direction is the angle that the wind is coming from (measured in degrees from North, clockwise).
# - yaw_angles is a list of current yaw angles for each turbine. 
# Return value:
# - None (this function does not contain a return statement)
# Makes use of: evaluate_single_direction_with_yaws

def greedy_optimize_yaw_angle_for_just_one_turbine(turbine_number, layout, wind_direction, yaw_angles) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

    # Hint: when debugging, placing the following print statement inside your loop my help you understand the calculations as they are performed       
    # print(f"Optimize turbine: {turbine_number}, yaw angles: {yaw_angles}, power: {power:.1f}")

In [ ]:
import code_analyser

layout08 = [(0.0, 0.0), (171.01007166283435, 469.8463103929542), (342.0201433256687, 939.6926207859084), (513.0302149885031, 1409.5389311788626), (492.403876506104, 86.8240888334652), (663.4139481689383, 556.6703992264195), (834.4240198317727, 1026.5167096193736), (1005.4340914946072, 1496.363020012328), (813.7976813493736, -296.1981327260238), (984.807753012208, 173.6481776669304), (1155.8178246750424, 643.4944880598846), (1326.8278963378766, 1113.340798452839), (1306.2015578554776, -209.3740438925586), (1477.2116295183118, 260.4722665003956), (1648.2217011811463, 730.3185768933497), (1819.2317728439807, 1200.164887286304)]
yaw_angles08 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4]
greedy_optimize_yaw_angle_for_just_one_turbine(15 , layout08, 225, yaw_angles08)
code_analyser.assert_equal(yaw_angles08, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'test6a')

In [ ]:
layout09 =  [(0.0, 0.0), (171.01007166283435, 469.8463103929542), (342.0201433256687, 939.6926207859084), (513.0302149885031, 1409.5389311788626), (492.403876506104, 86.8240888334652), (663.4139481689383, 556.6703992264195), (834.4240198317727, 1026.5167096193736), (1005.4340914946072, 1496.363020012328), (813.7976813493736, -296.1981327260238), (984.807753012208, 173.6481776669304), (1155.8178246750424, 643.4944880598846), (1326.8278963378766, 1113.340798452839), (1306.2015578554776, -209.3740438925586), (1477.2116295183118, 260.4722665003956), (1648.2217011811463, 730.3185768933497), (1819.2317728439807, 1200.164887286304)]
yaw_angles09 =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
greedy_optimize_yaw_angle_for_just_one_turbine(10 , layout09, 225, yaw_angles09)
code_analyser.assert_equal(yaw_angles09, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0], 'test6b')

In [ ]:
layout10 = [(0.0, 0.0), (171.01007166283435, 469.8463103929542), (342.0201433256687, 939.6926207859084), (513.0302149885031, 1409.5389311788626), (492.403876506104, 86.8240888334652), (663.4139481689383, 556.6703992264195), (834.4240198317727, 1026.5167096193736), (1005.4340914946072, 1496.363020012328), (813.7976813493736, -296.1981327260238), (984.807753012208, 173.6481776669304), (1155.8178246750424, 643.4944880598846), (1326.8278963378766, 1113.340798452839), (1306.2015578554776, -209.3740438925586), (1477.2116295183118, 260.4722665003956), (1648.2217011811463, 730.3185768933497), (1819.2317728439807, 1200.164887286304)]
yaw_angles10 = [0, -12, -12, 0, -12, -20, -12, 0, -16, 0, -16, 0, 0, 0, 0, 0]
greedy_optimize_yaw_angle_for_just_one_turbine(0 , layout10, 225, yaw_angles10)
code_analyser.assert_equal(yaw_angles10, [-12, -12, -12, 0, -12, -20, -12, 0, -16, 0, -16, 0, 0, 0, 0, 0], 'test6c')

In [ ]:
layout11 = [(0.0, 0.0), (171.01007166283435, 469.8463103929542), (342.0201433256687, 939.6926207859084), (513.0302149885031, 1409.5389311788626), (492.403876506104, 86.8240888334652), (663.4139481689383, 556.6703992264195), (834.4240198317727, 1026.5167096193736), (1005.4340914946072, 1496.363020012328), (813.7976813493736, -296.1981327260238), (984.807753012208, 173.6481776669304), (1155.8178246750424, 643.4944880598846), (1326.8278963378766, 1113.340798452839), (1306.2015578554776, -209.3740438925586), (1477.2116295183118, 260.4722665003956), (1648.2217011811463, 730.3185768933497), (1819.2317728439807, 1200.164887286304)]
yaw_angles11 = [-16, -12, -12, 0, -12, -12, 0, 20, -16, -12, -12, 0, 0, 0, 0, -4]
greedy_optimize_yaw_angle_for_just_one_turbine(15 , layout11, 225, yaw_angles11)
code_analyser.assert_equal(yaw_angles11, [-16, -12, -12, 0, -12, -12, 0, 20, -16, -12, -12, 0, 0, 0, 0, 0], 'test6d')

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_greedy_optimize_yaw_angle_for_just_one_turbine()

# In which order should we greedily optimize the yaw angles for each turbine

We could just apply the above greedy optimization algorithm to each of the turbines, first turbine number 1, then turbine number 2 and so on.

However, our heuristic will tend to work better if we apply it to each turbine using a more strategic order that takes accounnt of the wind direction. 

We will start by determining the greedy optimal angle for the turbine that is closest to the direction from which the wind is blowing, followed by the second closest and so on, before finally the most down wind  turbine. This order won't always be the same, it will depend on the wind direction.

For example if the wind is coming from the south west as in the following illustration, then turbine 0 will be processed first, followed by 4, then 8 and finally 11. Whereas if the wind was coming from the south, the turbine 8 would be processed first.

<img src="images/downwind.png">

<div style="background-color: #00cc00; padding:10px">

# Task 10: Compute the distance downwind for a particular turbine

To determine the distance downwind of a turbine, we need to project the location vector $(x,y)$ of the turbine onto the wind vector $\vec{w}$.

<img src="images/wind projection.png">

If the wind is from direction $\theta$ (measured clockwise from North), then the magnitude of the x and y components of the wind direction will be $\sin \theta$ and $\cos \theta$ respectively. However, the wind is *from* direction $\theta$, not *towards* direction $\theta$, so the (unit length) wind vector will be $(-\sin \theta, -\cos \theta)$. The scalar projection (i.e. magnitude) of the location vector onto the wind vector is just the dot product of these two vectors, which is $(-x \sin \theta, -y \cos \theta)$. 

Implement the following function to compute this downwind distance.
</div>

In [ ]:
import math  

# This function computes the downwind distance of the specified location, where the downwind distance is the scalar projection of the location vector onto the wind vector (see above formula)
# Parameters:
# - turbine_location is an (x,y) tuple corresponding to the location of one of the turbines 
# - wind_direction_degrees is the direction that the wind is blowing from, measured in degrees, clockwise from North.
# Return value:
# - a real/floating point number representing the downwind distance of this turbine 

def distance_downwind(turbine_location, wind_direction_degrees): # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

In [ ]:
import code_analyser

distance00 = distance_downwind((1, 1), 0)
code_analyser.assert_equal(distance00, -1, "test7a")

In [ ]:
distance01 = distance_downwind((1, 1), 1)
code_analyser.assert_equal(distance01, -1.017300101593675, "test7b")

In [ ]:
distance02 = distance_downwind((1, 1), 10)
code_analyser.assert_equal(distance02, -1.1584559306791384, "test7c")

In [ ]:
distance03 = distance_downwind((1, 1), 30)
code_analyser.assert_equal(distance03, -1.3660254037844388, "test7d")

In [ ]:
distance04 = distance_downwind((1, 1), 45)
code_analyser.assert_equal(distance04, -1.414213562373095, "test7e")

In [ ]:
distance05 = distance_downwind((1, 1), 90)
code_analyser.assert_equal(distance05, -1, "test7f")

In [ ]:
distance06 = distance_downwind((1, 1), 135)
code_analyser.assert_equal(distance06, 0, "test7g")

In [ ]:
distance07 = distance_downwind((1, 1), 180)
code_analyser.assert_equal(distance07, 1, "test7h")

In [ ]:
distance08 = distance_downwind((1, 1), 225)
code_analyser.assert_equal(distance08, 1.414213562373095, "test7i")

In [ ]:
distance09 = distance_downwind((1, 1), 270)
code_analyser.assert_equal(distance09, 1, "test7j")

In [ ]:
distance10 = distance_downwind((1, 1), 315)
code_analyser.assert_equal(distance10, 0, "test7k")

In [ ]:
distance11 = distance_downwind((1299.0381056766578, 1750.0), 225)
code_analyser.assert_equal(distance11, 2155.99552062015, "test7l")

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_distance_downwind()

# Sorting by distance

Given the distance downwind of each turbine, the following function (which has been provided for you) computes the order of the turbines, with more upwind turbines first and more downwind turbines later. For example if the downwind distance of each turbine is $[-4.53, 27.45, 5.35]$, then the resulting order of the turbine numbers would be $[0, 2, 1]$, i.e. turbine number $0$ first (because it is the most upwind turbine with a distance of $-4.53$, followed by turbine number $2$ (which has a downwind distance of $5.35$), followed by turbine number $1$ (which has a downwind distance of $27.45$.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

# This function computes the order of the turbines, with more upwind turbines first and more downwind turbines later.
# Parameters:
# - list_of_distances is a list of the downwind distance for each of the turbines (with the first corresponding to the first turbine, etc)
# Return value:
# - a list of the turbine numbers, sorted according to their downwind distances.

def sort_turbines_by_distance_downwind(list_of_distances) :
    def lookup(turbine_number) :
        return list_of_distances[turbine_number]
    list_of_turbine_numbers = range(len(list_of_distances)) 
    return sorted(list_of_turbine_numbers, key=lookup)

In [ ]:
import code_analyser

sorted01 = sort_turbines_by_distance_downwind([35, 25, 73])
code_analyser.assert_equal(sorted01, [1, 0, 2])

In [ ]:
sorted02 = sort_turbines_by_distance_downwind([0.0, 482.9629131445341, 612.3724356957945, 1095.3353488403286, 353.5533905932737, 836.5163037378079, 965.9258262890683, 1448.8887394336023, 707.1067811865474, 1190.0696943310816, 1319.479216882342, 1802.442130026876, 1060.6601717798212, 1543.6230849243555, 1673.0326074756158, 2155.99552062015])
code_analyser.assert_equal(sorted02, [0, 4, 1, 2, 8, 5, 6, 12, 3, 9, 10, 7, 13, 14, 11, 15])

<div style="background-color: #00cc00; padding:10px">
    
# Task 11: Order by downwind distance

The following function (which you need to implement) uses the above two functions to return a list of the turbine numbers sorted according to their distance downwind.
</div>

In [ ]:
# This functions returns a list of the turbine numbers sorted according to their downwind distance.
# The two functions above are used to compute each downwind distance and to sort the turbine numbers by downwind distance.
# Parameters:
# - layout is a list of (x,y) tuples representing the location of each turbine
# - wind_direction the angle from which the wind is blowing measured in degrees clockwise from North
# Return value:
# - a list of the turbine numbers sorted according to their downwind distance (with the most upwind turbines listed first and the most downwind turbines last)
# Makes use of: distance_downwind, sort_turbines_by_distance_downwind

def orderby_downwind(layout, wind_direction) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

In [ ]:
layout12 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
visualize_layout(layout12)

In [ ]:
import code_analyser

layout13 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
order13 = orderby_downwind(layout13, 0) # wind from the north
code_analyser.assert_equal(order13, [8, 5, 2, 7, 4, 1, 6, 3, 0], 'test8a')

In [ ]:
layout14 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
order14 = orderby_downwind(layout14, 45) # wind from the north east
code_analyser.assert_equal(order14, [8, 7, 5, 6, 4, 2, 3, 1, 0], 'test8b')

In [ ]:
layout15 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
order15 = orderby_downwind(layout15, 90) # wind  from the east
code_analyser.assert_equal(order15, [6, 7, 8, 3, 4, 5, 0, 1, 2], 'test8c')

In [ ]:
layout16 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
order16 = orderby_downwind(layout16, 135) # wind  from the south east
code_analyser.assert_equal(order16,  [6, 3, 7, 0, 4, 8, 1, 5, 2], 'test8d')

In [ ]:
layout17 = [(-281.2819,-316.7150),(375.2950,-88.8308),(-75.2815,-182.2680),(316.1102,-283.3000),(264.2196,-301.3396),(-348.4140,-102.1951),(-346.5306,219.1108),(-194.5401,313.0611),(91.8581,-123.0207),(-448.4288,442.9667)]
order17 = orderby_downwind(layout17, 135) # wind  from the south east
code_analyser.assert_equal(order17,  [3, 4, 1, 8, 2, 0, 5, 7, 6, 9], 'test8e')

In [ ]:
layout18 = [(-281.2819,-316.7150),(375.2950,-88.8308),(-75.2815,-182.2680),(316.1102,-283.3000),(264.2196,-301.3396),(-348.4140,-102.1951),(-346.5306,219.1108),(-194.5401,313.0611),(91.8581,-123.0207),(-448.4288,442.9667)]
order18 = orderby_downwind(layout18, 315) # wind  from the north west
code_analyser.assert_equal(order18,  [9, 6, 7, 5, 0, 2, 8, 1, 4, 3], 'test8f')

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_orderby_downwind()

<div style="background-color: #00cc00; padding:10px">

# Task 12: Optimize all yaw angles

We've now established the order in which we optimize the yaw angle of each turbine and we have implemented a function that will greedily optimize the yaw angle for a specific turbine, so we can now combine the two to create the function below (which you need to implement) that will start with all turbines have a yaw angle of zero and then one by one greadily optimize the yaw angle for each turbine until we have an optimized list of yaw angles for each turbine.
</div>

In [ ]:
# In this function, we start with a yaw angle of zero degrees for each turbine and then one by one (with the upwind turbines processed first) 
# greedily select/update the optimal yaw angle for each turbine using the functions implemented above.
# Parameters:
# - layout is a list of (x,y) tuples representing the location of each turbine
# - wind_direction the angle from which the wind is blowing measured in degrees clockwise from North
# Return value:
# - a list of the optimal yaw angle (measured in degrees) for each turbine
# Makes use of: zero_yaw, orderby_downwind, greedy_optimize_yaw_angle_for_just_one_turbine

def optimize_all_yaw_angles(layout, wind_direction) : # do not change this line in any way - the function name and parameters must remain exactly as specified here
    return None # replace this line with your code for this function

In [ ]:
import code_analyser

layout19 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
best_angles19 = optimize_all_yaw_angles(layout19, 135) # wind from south east
code_analyser.assert_equal(best_angles19, [0, 0, 0, -16, 0, 0, -16, 0, 0], 'test9a')

In [ ]:
layout20 = [(0.0, 0.0), (-129.40952255126038, 482.9629131445342), (-258.81904510252076, 965.9258262890684), (482.9629131445342, 129.40952255126038), (353.5533905932738, 612.3724356957946), (224.14386804201342, 1095.3353488403288), (965.9258262890684, 258.81904510252076), (836.516303737808, 741.781958247055), (707.1067811865476, 1224.7448713915892)]
best_angles20 = optimize_all_yaw_angles(layout20, 225) # wind from south west
code_analyser.assert_equal(best_angles20, [-16, -16, 0, 0, 0, 0, 0, 0, 0], 'test9b')

In [ ]:
import code_analyser
code_analyser.test_non_functional_requirements_optimize_all_yaw_angles()

# Measuring and Visualizing the improvement from Wake Steering

The following function (which has been provided for you) will visually show us wakes before and after wake steering optimization and report the resulting percentage improvement in overall wind farm power.

In [ ]:
# The following function has been provided for you,  do not modify it in any way.

import matplotlib

# This function will visually show us wakes before and after wake steering optimization and report the resulting percentage improvement in overall wind farm power.
# Parameters:
# - layout is a list of (x,y) coordinates for each of the turbines
# - wind_direction is the angle from which the wind is blowing measured in degrees clockwise from North.
# Return value:
# - None
# Makes use of: zero_yaw, optimize_all_yaw_angles, visualize_flow_with_yaws, matplotlib

def visualize_before_and_after_yaw_optimization(layout, wind_direction):
    fig, ax = matplotlib.pyplot.subplots(1, 2, figsize=(20,10))
    
    default_yaw_angles = zero_yaw(layout)
    best_yaw_angles = optimize_all_yaw_angles(layout, wind_direction)

    power_before = visualize_flow_with_yaws(layout, wind_direction, default_yaw_angles, ax[0])
    ax[0].set_title(f'Yaw angles all 0, power = {power_before:.1f}')
    
    power_after = visualize_flow_with_yaws(layout, wind_direction, best_yaw_angles, ax[1])
    ax[1].set_title(f'Yaw angles = {best_yaw_angles}, power = {power_after:.1f}')

    improvement = 100 * (power_after-power_before)/power_before
    fig.suptitle(f'Wind direction = {wind_direction}, Improvement from wake steering = {improvement:.2f}%')

In [ ]:
# Create a 3 x 3 grid and rotate optimally 

grid_3x3b = create_grid_layout(3, 3, 500)
best_grid_3x3b = find_best_rotation(grid_3x3b)

In [ ]:
# Test when wind is coming from the North for our best rotated 3x3 grid

if best_grid_3x3b :
    visualize_before_and_after_yaw_optimization(best_grid_3x3b, 0)

In [ ]:
# Test when wind is coming from the South East for our best rotated 4x4 grid
if best_grid_3x3b :
    visualize_before_and_after_yaw_optimization(best_grid_3x3b, 135)

<div style="background:#00cc00; padding:10px">
    
# Automated feedback and semi-automated marking

Running the following code will give you a good idea how well you are meeting the assessment criteria and the rough mark that you can expect to receive. Note that some of the assessment criteria are marked manually by human tutors, so you won't know your final mark until it is officially submitted and assessed.
</div>

In [ ]:
# This code has been supplied to provide automated feedback and semi-automated marking (do not change it!)

import code_analyser
code_analyser.summary_feedback()